In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# DB 연결
import pymysql

# 실시간 주식가격 데이터
import FinanceDataReader as fdr

# 텍스트 분석
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from konlpy.tag import Okt
from gensim import models

# from wordcloud import WordCloud

# 모델 학습 및 평가
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score

# 기타
import datetime
from collections import Counter
import sys
import warnings
warnings.filterwarnings('ignore')

In [2]:
df_all_lgc = pd.read_csv('./data/전처리 전 데이터/df_youtube_삼프로LG화학.csv')

In [3]:
df_all_lgc.head()

,st_n,st_cd,ch_nm,date,title,text,views,length,description,url
0,LG화학,51910,삼프로TV_경제의신과함께,2021053100,매도 리포트에 흔들리는 LG화학.. 현실적 판단 VS 가치 훼손?_오늘아침 page...,으[음악]tera term[음악]으 으 으 으 으[음악]이름은 하나인데 직업은 서먹...,302650,7335,삼프로TV 주식대학 '봄학기'\n? 염블리의 친절한 주식 기초 교육 / 미주미 장우...,https://www.youtube.com/watch?v=rz_knoIe-fQ
1,LG화학,51910,삼프로TV_경제의신과함께,2020091700,LG화학 물적분할.. 그룹가치 상승이냐 주주가치 하락이냐 f.염승환 차장,구독과 좋아요는 사람입니다자 염치 염치 영차 차 우리 염성 안 차장에 보시겠습니다 ...,97309,2117,■page2 강의 신청하기\n- 사경인의 실전투자 아카데미 '리마스터'\nhttps...,https://www.youtube.com/watch?v=BrR0O6KQ5P0
2,LG화학,51910,삼프로TV_경제의신과함께,2020091700,LG화학의 물적분할.. 주가 하락을 걱정하는 이유는? f.김현수 선임연구원,좋아요 구독 눌러주세요자 그럼 오늘 첫 번째 뉴스 부터 시작을 해 봐야죠먼저 뭐 미...,52967,1321,■page2 강의 신청하기\n- 사경인의 실전투자 아카데미 '리마스터'\nhttps...,https://www.youtube.com/watch?v=bXAbIIeVD0U
3,LG화학,51910,삼프로TV_경제의신과함께,2020101200,'배터리 셀 분리막 결함' 발표.. 현대차 VS LG화학 전쟁 발발하나? f.권순우 기자,자 그러면 오늘 노려 일이니까요 발칙하게 한번 가보겠습니다뭐 리토의 방송에 건수를 ...,31501,1988,■ [삼프로TV 시청자 자문단 1기 응모 양식] 링크\nhttps://forms.g...,https://www.youtube.com/watch?v=3cHrONGe5gs
4,LG화학,51910,삼프로TV_경제의신과함께,2021011100,LG화학의 '2차 전지 IPO'.. 생각보다 이른 시기에 상장할 수도 있다? f.이...,산업을 이해하고 기업을 이해해야 돈의 흐름이 보입니다리서치 명과 메리츠 증권과 3%...,369589,2155,"■삼프로TV가 준비한 다양한 프리미엄 콘텐츠\n\n- 삼프로 TV 주식대학 ""업종 ...",https://www.youtube.com/watch?v=0HFXP_rtYPw


In [4]:
df_all_lgc[df_all_lgc['date'] == 2020091700]

,st_n,st_cd,ch_nm,date,title,text,views,length,description,url
1,LG화학,51910,삼프로TV_경제의신과함께,2020091700,LG화학 물적분할.. 그룹가치 상승이냐 주주가치 하락이냐 f.염승환 차장,구독과 좋아요는 사람입니다자 염치 염치 영차 차 우리 염성 안 차장에 보시겠습니다 ...,97309,2117,■page2 강의 신청하기\n- 사경인의 실전투자 아카데미 '리마스터'\nhttps...,https://www.youtube.com/watch?v=BrR0O6KQ5P0
2,LG화학,51910,삼프로TV_경제의신과함께,2020091700,LG화학의 물적분할.. 주가 하락을 걱정하는 이유는? f.김현수 선임연구원,좋아요 구독 눌러주세요자 그럼 오늘 첫 번째 뉴스 부터 시작을 해 봐야죠먼저 뭐 미...,52967,1321,■page2 강의 신청하기\n- 사경인의 실전투자 아카데미 '리마스터'\nhttps...,https://www.youtube.com/watch?v=bXAbIIeVD0U
26,LG화학,51910,삼프로TV_경제의신과함께,2020091700,"연준, AIT 도입 결정 후 첫 FOMC 회의.. 내용은 무난했다?_오늘아침 pag...",놈 5으 으[음악]김과장 경기가 불어날 때 어떻게 투자해이럴땐 든 들은 안전 자산의...,154462,6554,00:00 오프닝\n\n06:06 뉴스3 (긴급 인터뷰 - 김현수 선임연구원)\nL...,https://www.youtube.com/watch?v=8G1254oRO3g
54,LG화학,51910,삼프로TV_경제의신과함께,2020091700,"시장에 대한 연준의 솔직한 고백.. ""우리는 잠들지 않아"" f.이효석 팀장",좋아요 구독 눌러주세요하도 채팅 장 의 lg 화학이에 대한 에 뭐라고 할까요 송 토...,29601,2636,■page2 강의 신청하기\n- 사경인의 실전투자 아카데미 '리마스터'\nhttps...,https://www.youtube.com/watch?v=iVYZ9fe23IY
58,LG화학,51910,삼프로TV_경제의신과함께,2020091700,"이효석 팀장에게 듣는 9월 FOMC 미리보기_퇴근길 page2_이효석, 김현겸",으222[음악]으 으 으 으 으 으[음악]김과장 경기가 불안할 때 어떻게 투자해이럴...,103802,4172,00:00 오프닝\n\n03:59 증시 셔터맨 (김현겸 과장)\n상승에 대한 피로...,https://www.youtube.com/watch?v=vzGoEhru_5M
133,LG화학,51910,삼프로TV_경제의신과함께,2020091700,"골드만삭스가 말하는 10%이상 성장할 10가지 종목_글로벌 라이브_박광남, 김철중,...",[음악][박수][음악]내 안녕하십니까 아 투자 공부를 빙자한 온라인 해외 치가 입니...,97621,5805,00:00 오프닝\n\n04:18 글로벌 마켓 브리핑 - 미래에셋대우 박광남 선임연...,https://www.youtube.com/watch?v=Zmn0D5MqggM
149,LG화학,51910,삼프로TV_경제의신과함께,2020091700,과거 디스플레이 사례로 보는 배터리 산업의 미래,아 김철중 한국이란 모시고 배터리 이야기좀 들어보겠습니다어서 오십시오 안녕하세요 4...,30063,2107,■page2 강의 신청하기\n- 사경인의 실전투자 아카데미 '리마스터'\nhttps...,https://www.youtube.com/watch?v=sOW1CiuDsEg


In [5]:
df_all_lgc[df_all_lgc['date'] == 2020091700].shape

(7, 10)

In [6]:
# description, url 제외
df_lgc = df_all_lgc.iloc[:, :8]

In [7]:
df_lgc.head()

,st_n,st_cd,ch_nm,date,title,text,views,length
0,LG화학,51910,삼프로TV_경제의신과함께,2021053100,매도 리포트에 흔들리는 LG화학.. 현실적 판단 VS 가치 훼손?_오늘아침 page...,으[음악]tera term[음악]으 으 으 으 으[음악]이름은 하나인데 직업은 서먹...,302650,7335
1,LG화학,51910,삼프로TV_경제의신과함께,2020091700,LG화학 물적분할.. 그룹가치 상승이냐 주주가치 하락이냐 f.염승환 차장,구독과 좋아요는 사람입니다자 염치 염치 영차 차 우리 염성 안 차장에 보시겠습니다 ...,97309,2117
2,LG화학,51910,삼프로TV_경제의신과함께,2020091700,LG화학의 물적분할.. 주가 하락을 걱정하는 이유는? f.김현수 선임연구원,좋아요 구독 눌러주세요자 그럼 오늘 첫 번째 뉴스 부터 시작을 해 봐야죠먼저 뭐 미...,52967,1321
3,LG화학,51910,삼프로TV_경제의신과함께,2020101200,'배터리 셀 분리막 결함' 발표.. 현대차 VS LG화학 전쟁 발발하나? f.권순우 기자,자 그러면 오늘 노려 일이니까요 발칙하게 한번 가보겠습니다뭐 리토의 방송에 건수를 ...,31501,1988
4,LG화학,51910,삼프로TV_경제의신과함께,2021011100,LG화학의 '2차 전지 IPO'.. 생각보다 이른 시기에 상장할 수도 있다? f.이...,산업을 이해하고 기업을 이해해야 돈의 흐름이 보입니다리서치 명과 메리츠 증권과 3%...,369589,2155


In [8]:
# 출처 '자체제작 감성어휘사전 모듈_ver2.ipynb'

def set_data(df,com_num = 0):  # panda, numpy, datetime, FinanceDataReader, konlpy, Counter
    # 일, 시간 나누기
    df['date'] = df['date'].astype(str)
    df['date_d'] = df['date'].str[:-2]
    df['date_h'] = df['date'].str[-2:]
    # 타입을 데이트 타입으로 만듬
    df['date_d'] = pd.to_datetime(df['date_d'])
    
    df = df.sort_values(by='date_d') # 일기준으로 오름차순 정렬
    
    if com_num != 0:
    #     통합 데이터 활용시 업종코드 지정
        df = df[df['st_cd'] == com_num] # 해당 회사만 추출
        num = str(com_num).zfill(6) # 종목코드를 6자리로 만들어줌
        
    else:
        # 특정 업종만 할 때
        num = str(df['st_cd'].iloc[0]).zfill(6)
    
    # 널값 제거
    df = df.dropna()
    
    # ============== 날짜 조정
    ## 전일 15시 ~ 금일 15시로 날짜 조정
    after_market = ['15', '16', '17', '18', '19', '20', '21', '22', '23']
    
    for i in range(len(df)):
        if df['date_h'].iloc[i] in after_market:
            df['date_d'].iloc[i] += datetime.timedelta(1)
    
    # 주말 및 공휴일 데이터
    ### Holidays
    try:
        # 서버가 열려있을 때
        db = pymysql.connect(user='root',
                             passwd='1234',
                             host='3.35.70.166',
                             db='proj',
                             charset='utf8')

        cursor = db.cursor(pymysql.cursors.DictCursor)

        sql = "select * from holidays"
        cursor.execute(sql)
        result = cursor.fetchall()
    
        # DataFrame으로 변경
        holi = pd.DataFrame(result)
        # db 닫기 --> 안하면 메모리 잡아먹음
        db.close()
    except:
        # 서버 없을 때 깃허브에서 바로 가져옴
        db_holi = 'https://raw.githubusercontent.com/chaerui7967/stock_predict_news_and_youtube/master/감성분석/data/holidays.csv'
        holi = pd.read_csv(db_holi)
    
    # date 컬럼을 날짜 형식으로 변경
    holi['date'] = pd.to_datetime(holi['date'])
    
    
    ### ===================주말 및 공휴일 제외
    
    ## 주말 및 공휴일만 추출
    market_closed = holi[holi['holiday']=="O"].reset_index(drop=True)
    
    ## 휴장일 List 생성
    market_closed_list = list(market_closed['date'])
    
    # 주말 및 공휴일 제외
    while len(df[df['date_d'].isin(market_closed_list)]['date_d']) !=0:
        for i in df[df['date_d'].isin(market_closed_list)]['date_d'].index:
            df['date_d'][i] += datetime.timedelta(1)
     
    
    
    # 주식 데이터 가져오기
    start_date = str(df['date_d'].iloc[0])[:10].replace('-', '')
    end_date = str(df['date_d'].iloc[-1])[:10].replace('-', '')  # 뉴스 데이터 기준 최근 값까지의 주식 데이터
    
    stock = fdr.DataReader(num, start = start_date, end = end_date)
    
    # change 컬럼 판단 기준 close에서 수수료 보다 많은 비율이 올랐으면 1, 떨어졌으면 -1, 그 사이 0
    point = stock['Close'] * stock['Change']
    spli = stock['Close'] * 0.001  # 수수료가 0.1 % 라고 할 때
    
    stock['ud'] = np.where( point > spli, 1,
                          np.where(point < spli, -1, 0))
    
    stock = stock.reset_index() # 날짜 컬럼 가져오기
    
    s_1 = stock[['Date','ud']] # 필요한 컬럼만 가져오기
    
    s_1.columns = ['date_d','ud'] # 컬럼 명 초기화
    
    # 뉴스데이터와 주식데이터 merge
    df_1 = pd.merge(df,s_1,on='date_d')  # 영향받는 컬럼 기준으로 inner 조인 
    
    return df_1

In [9]:
dataSet = set_data(df_lgc)
dataSet.head()

,st_n,st_cd,ch_nm,date,title,text,views,length,date_d,date_h,ud
0,LG화학,51910,삼프로TV_경제의신과함께,2019090100,주식 투자자가 버려야 할 7가지! [최준철 대표의 주식 FM가이드 #3],[음악]최진철 테스의 주식 fm 가이드세번째 시간이 시작됐습니다 반갑습니다 점프로 ...,38118,4364,2019-09-02,00,-1
1,LG화학,51910,삼프로TV_경제의신과함께,2019090300,"누가 마지막까지 투자하는가!_19.09.03_김효진,곽상준 [오늘아침 Page2]",4[음악]깊이가 나는 경제 컨텐츠 오늘 아침 페이지 투 정용진 김도완 입니다[음악]...,13446,4244,2019-09-03,00,1
2,LG화학,51910,삼프로TV_경제의신과함께,2019110300,주식 포트폴리오 운용 방법! 경마 분석법(?) [최준철 대표의 주식 FM가이드 05-1],5 으주식 시장의 야전사령관 최준철 대표 함께하는 주식 sm 카이드 다섯번째시간이 ...,26233,3240,2019-11-04,00,1
3,LG화학,51910,삼프로TV_경제의신과함께,2019110800,"삼성의 스마트폰 전략, 변화의 이유는? _19.11.07_박종일,박제영 [퇴근길 P...",깨알 페이지 투4 안녕하십니까 11월 7일 퇴근 낄 페이지 툰 관세 장벽을 일부 허...,11245,4587,2019-11-08,00,1
4,LG화학,51910,삼프로TV_경제의신과함께,2020011800,[심층인터뷰] 반복되는 4분기 '어닝쇼크' 이유는?,[음악]오늘 어또 손님 한번 보셔서 공부해야 줘 요 한테 증권의 김광현 선임 년 무...,6132,1307,2020-01-20,00,1


In [10]:
dataSet.title[0]

'주식 투자자가 버려야 할 7가지!  [최준철 대표의 주식 FM가이드 #3]'

In [11]:
len(dataSet.text[0])

32302

In [12]:
dataSet.text[0]

'[음악]최진철 테스의 주식 fm 가이드세번째 시간이 시작됐습니다 반갑습니다 점프로 구요 4 2% 입니다불탄 fm 과 이때도 뭔지 아는 거예요아 마켓의 있어 잘라서 야짤 이것 역시 를 거는 2분야 뭐 하하 빠른거아냐 그거아 그렇게 좀 출연자들에게 가만 있지 못하니까아 빌라도 지원 얘기가 계속 나오네요 우정을 하루 4시간으로는 세번째 씨가 에 방문으로 이제 두 번째가 그 정 그런거는 정프로가 알아서 합니다 어제 그제 위 제로 생각했네그거 해 가지 안 하면 아 여튼 밝았습니다 우리 처지 설 대표님예 스탐 장장 하셔서 예뻐 같습니다 4 우리조 주식fm 가이드 이란 게 이제 쉴드 매뉴얼 뭐그날 조범 이렇게 챙겨 범 될지도 모르겠소 서울 fm 에fm 이라는 가족용 아 왜 음 저는 그xls 아니겠습니까 그러다 보니까 야전 믿은 게 없어요그래 예 군 테 전 이제 공부 내는딜도 매뉴얼 이런 게 없었어요 사실 에 했는데 전 프로가 모르 거지당신은 저 할 줄 안다고 15 줄이 저촉 뭐 요분들도 총무가 있나 4 있는거예요아 지금 고은비 하시게 달 초 어떻게 7 금 특기가 있어요 은 총무는무슨 이래요공군 총무 유천은 식당에도 좀 있었구요그 다음에 줘 식당에서 부분이 되요 boq 관사 관리도 식당에서 밥 앞에쭉 류큐 관리 병 이로써 이렇게도 관리하고 야 아 내 표기 있었잖아적이며 안되니까 장교 숙소 비어캔 204 하겠어 아주 이야기가 잠겨서결과 아니에요 그 이 빚자 그 독신자 단테 줘 아 아 아 그래요에바 텔러 에어 q 로 뭐야 도대체 그 q 는 뭘까야 우리 세 글자 주기 아마 예 아 오늘 노브 겠지4 여사해 떠오르게 딱히 뭐 하고 이제 우체국에서 들음 있었고 그 만큼그 생각하면서 이렇게 여러분들이 있었다구요그래서 되게 여러분들 이발 됐어 심지어 이 발병 에 등 단기적 못했구나1 제가 고용 안되겠네가요 내 사랑 합니다 아 알투비 르 베제라 운영 등 병장 정말 재고인드라 굿은 많이 하셨네너를 데려가 열매 작업이 싸게 입이 부서져 고서 에서 그 굉장히 그치 아된경 계획 아직 으그래서 이

In [13]:
dataSet.shape

(507, 11)

In [14]:
sample = dataSet.head()
sample

,st_n,st_cd,ch_nm,date,title,text,views,length,date_d,date_h,ud
0,LG화학,51910,삼프로TV_경제의신과함께,2019090100,주식 투자자가 버려야 할 7가지! [최준철 대표의 주식 FM가이드 #3],[음악]최진철 테스의 주식 fm 가이드세번째 시간이 시작됐습니다 반갑습니다 점프로 ...,38118,4364,2019-09-02,00,-1
1,LG화학,51910,삼프로TV_경제의신과함께,2019090300,"누가 마지막까지 투자하는가!_19.09.03_김효진,곽상준 [오늘아침 Page2]",4[음악]깊이가 나는 경제 컨텐츠 오늘 아침 페이지 투 정용진 김도완 입니다[음악]...,13446,4244,2019-09-03,00,1
2,LG화학,51910,삼프로TV_경제의신과함께,2019110300,주식 포트폴리오 운용 방법! 경마 분석법(?) [최준철 대표의 주식 FM가이드 05-1],5 으주식 시장의 야전사령관 최준철 대표 함께하는 주식 sm 카이드 다섯번째시간이 ...,26233,3240,2019-11-04,00,1
3,LG화학,51910,삼프로TV_경제의신과함께,2019110800,"삼성의 스마트폰 전략, 변화의 이유는? _19.11.07_박종일,박제영 [퇴근길 P...",깨알 페이지 투4 안녕하십니까 11월 7일 퇴근 낄 페이지 툰 관세 장벽을 일부 허...,11245,4587,2019-11-08,00,1
4,LG화학,51910,삼프로TV_경제의신과함께,2020011800,[심층인터뷰] 반복되는 4분기 '어닝쇼크' 이유는?,[음악]오늘 어또 손님 한번 보셔서 공부해야 줘 요 한테 증권의 김광현 선임 년 무...,6132,1307,2020-01-20,00,1


## Stop Words List

In [15]:
# Update 계속 하는 중!
media = ['오늘', '년도', '이제', '요즘', '지금', '우리나라', '한번', '사실',  \
         '증시', '개장', '얘기', '이렇다', '참고', '없다', '때문', \
         '아니다', '어서', '오십', '시오', '모습', '내용', '정도', '그렇다', '어쨌든', '어떻다', \
         '박지영', '미래에셋', '어머님', '음악']

In [16]:
# 출처 '자체제작 감성어휘사전 모듈_ver2.ipynb' > def giro_dic
##표시 수정 부분

def youtube_pre(df,num = 0):
    
## media 전역 변수 선언
#     global media
    
    df = set_data(df, num)
    print('데이터 셋 구성 완료')
    
    okt = Okt()
    print('okt 모듈 활성')
    
    
    # 데이터 전처리
    # =================== nlp
    # 1. 제목, 텍스트 --> 명사, 형용사 추출
    # 태그 삭제        
    # \n \t 삭제
    df['text'] = df['text'].str.replace('[\n|\t|\r]','')
#     df['text'] = df['text'].str.replace('\s','')
    df['text'] = df['text'].str.replace('[^a-zA-Z0-9가-힣]','')
    
    # 토큰화 및 품사 태깅
    df['Contents_input'] = df['text'].apply(lambda x: okt.pos(x, norm=True, stem=True))
    print('태그 삭제 및 품사 태깅 완료')
    
    # 명사, 형용사 추출
    for k in range(len(df['Contents_input'])):
        wordList = []
        for j in range(len(df['Contents_input'][k])):
            if df['Contents_input'][k][j][1] == 'Noun' or df['Contents_input'][k][j][1] == 'Adjective':
                wordList.append(df['Contents_input'][k][j][0])         
        df['Contents_input'][k] = wordList
    print('명사, 형용사 추출')
    
    # 2. 불용어 처리
    # 불용어 사전 로딩
##     url = 'https://raw.githubusercontent.com/chaerui7967/stock_predict_news_and_youtube/master/%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D/data/stopwords_ver1.txt'
    url = 'https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt'
    stopwords = list(pd.read_csv(url, header=None)[0])
##     media = ['매일경제','mkcokr','무단전재및재배포금지','무단전재','재배포금지']
#     media = ['오늘', '년도', '이제', '요즘', '지금', '우리나라', '한번', '사실', '처럼', \
#              '증시', '개장', '얘기', '이렇다', '참고', '없다', '때문', '아니다', \
#              '안녕하십니까', '안녕하세요우', '안녕하다', '어서', '오십', '시오', '모습', '내용', '정도', \
#              '그렇다', '어쨌든', '어떻다', '박지영', '미래에셋', '어머님', '음악']
    print('불용어 사전 로딩..')
    
    
    for i in range(len(df['Contents_input'])):
        for word in df['Contents_input'][i]:
            if word in stopwords:
                df['Contents_input'][i].remove(word)
            if word in media:
                df['Contents_input'][i].remove(word) 
    print('불용어 처리 완료')
    
    # 단어길이가 1이하인 단어 제거
    for index, wordList in enumerate(df['Contents_input']):
#         if len(df['Contents_input'][index]) == 0:
#             continue
        try:
            for word in wordList:
                if len(word) <= 1:
                    df['Contents_input'][index].remove(word)
        except:
            continue
            
    print('단어 길이 1이하인 단어 제거')

    return df

In [44]:
'''
# 단어 길이 1 이하 제거 안 되는 경우가 있음

for index, wordList in enumerate(sample_clean['Contents_input']):
    print(index, wordList)
    if len(sample_clean['Contents_input'][index]) == 0:
        continue
    for word in wordList:
        if len(word) <= 1:
            sample_clean['Contents_input'][index].remove(word)

sample_clean['Contents_input'][1] # 1 index '말'
'''

## Text-Preprocessing

In [56]:
# df_clean = youtube_pre(df)
# df_clean.head()

### Sample

In [17]:
%%time 

# 5개 샘플 데이터
sample_clean = youtube_pre(sample)

데이터 셋 구성 완료
okt 모듈 활성
태그 삭제 및 품사 태깅 완료
명사, 형용사 추출
불용어 사전 로딩..
불용어 처리 완료
단어 길이 1이하인 단어 제거
Wall time: 10min 24s


In [18]:
sample_clean

,st_n,st_cd,ch_nm,date,title,text,views,length,date_d,date_h,ud_x,ud_y,Contents_input
0,LG화학,51910,삼프로TV_경제의신과함께,2019090100,주식 투자자가 버려야 할 7가지! [최준철 대표의 주식 FM가이드 #3],음악최진철테스의주식fm가이드세번째시간이시작됐습니다반갑습니다점프로구요42입니다불탄fm...,38118,4364,2019-09-02,00,-1,-1,"[최진철, 테스, 주식, 가이드, 세번, 시작, 반갑다, 점프, 이다, 불타다, 마..."
1,LG화학,51910,삼프로TV_경제의신과함께,2019090300,"누가 마지막까지 투자하는가!_19.09.03_김효진,곽상준 [오늘아침 Page2]",4음악깊이가나는경제컨텐츠오늘아침페이지투정용진김도완입니다음악49월3일화요일아침오너스페...,13446,4244,2019-09-03,00,1,1,"[깊이, 가나, 경제, 컨텐츠, 오늘아침, 페이지, 정용진, 완, 이다, 오른쪽, ..."
2,LG화학,51910,삼프로TV_경제의신과함께,2019110300,주식 포트폴리오 운용 방법! 경마 분석법(?) [최준철 대표의 주식 FM가이드 05-1],5으주식시장의야전사령관최준철대표함께하는주식sm카이드다섯번째시간이시작되었습니다안녕하세...,26233,3240,2019-11-04,00,1,1,"[주식시장, 야전, 사령관, 최준, 대표, 주식, 이드, 시간, 시작, 안녕하다, ..."
3,LG화학,51910,삼프로TV_경제의신과함께,2019110800,"삼성의 스마트폰 전략, 변화의 이유는? _19.11.07_박종일,박제영 [퇴근길 P...",깨알페이지투4안녕하십니까11월7일퇴근낄페이지툰관세장벽을일부허무는합의가도출됐다는속보가...,11245,4587,2019-11-08,00,1,1,"[깨알, 페이지, 안녕하십니까, 국가, 식품, 클러스터, 스품, 관련, 스타트업, ..."
4,LG화학,51910,삼프로TV_경제의신과함께,2020011800,[심층인터뷰] 반복되는 4분기 '어닝쇼크' 이유는?,음악오늘어또손님한번보셔서공부해야줘요한테증권의김광현선임년무한을모시고오늘은4분기실적에대...,6132,1307,2020-01-20,00,1,1,"[손님, 공부, 증권, 김광현, 선임, 무한, 모시, 주저, 잠정, 실적, 발표, ..."


In [21]:
sample_clean.Contents_input[0]

['최진철',
 '테스',
 '주식',
 '가이드',
 '세번',
 '시작',
 '반갑다',
 '점프',
 '이다',
 '불타다',
 '마켓',
 '라서',
 '역시',
 '주식',
 '가이드',
 '게이',
 '쉴드',
 '매뉴얼',
 '그날',
 '조범',
 '서울',
 '가족',
 '왜음',
 '야전',
 '전이',
 '제공',
 '부내',
 '딜도',
 '매뉴얼',
 '프로',
 '저촉',
 '뭐요',
 '총무',
 '아지',
 '금고',
 '은비',
 '달초',
 '특기',
 '은총',
 '공군',
 '총무',
 '유천',
 '식당',
 '좋다',
 '그다음',
 '식당',
 '부분',
 '관사',
 '관리',
 '식당',
 '밥앞',
 '류큐',
 '관리',
 '관리',
 '아내',
 '표기',
 '안되다',
 '장교',
 '숙소',
 '비어',
 '이야기',
 '결과',
 '독신자',
 '에바',
 '텔러',
 '에어',
 '로뭐',
 '도대체',
 '리세',
 '글자',
 '노브',
 '여사하다',
 '딱하다',
 '우체국',
 '생각',
 '이발',
 '발병',
 '등단',
 '기적',
 '고용',
 '안되다',
 '사랑',
 '알투',
 '비르',
 '제라',
 '운영',
 '병장',
 '정말',
 '재고',
 '인드라',
 '열매',
 '작업',
 '굉',
 '장하다',
 '치아',
 '계획',
 '래서',
 '쉴드',
 '매뉴얼',
 '전교',
 '처음',
 '자드',
 '전교',
 '이유',
 '주식시장',
 '굉장하다',
 '어렵다',
 '투자',
 '나이',
 '고민',
 '둘째',
 '도의',
 '고민',
 '어렵다',
 '시기',
 '차례',
 '실전',
 '모시',
 '좋다',
 '좋다',
 '달라',
 '컨셉',
 '상승',
 '교문',
 '제안',
 '마음가짐',
 '래서',
 '주식',
 '투자',
 '대하',
 '자세',
 '쉔',
 '이설',
 '대표',
 '모시',
 '이야기',
 '종가',
 '현재',
 '필요하다'

In [22]:
sample_clean.Contents_input[1] # 한 글자 안 지워짐

['깊이',
 '가나',
 '경제',
 '컨텐츠',
 '오늘아침',
 '페이지',
 '정용진',
 '완',
 '이다',
 '오른쪽',
 '어스',
 '페이지',
 '라이브',
 '일이',
 '항상',
 '오지',
 '김효진',
 '연구원',
 '흥미롭다',
 '주제',
 '계속',
 '해주시',
 '어제',
 '새벽',
 '새벽',
 '좋다',
 '자정',
 '뭐총',
 '문화',
 '제도',
 '광고',
 '뉴스',
 '시작',
 '도록',
 '히터',
 '아빠',
 '발던밥',
 '두껍다',
 '새콤',
 '뉴스',
 '잠좀',
 '먼저',
 '국내',
 '배터리',
 '업계',
 '화주',
 '화학',
 '이노베이션',
 '송전',
 '사이',
 '중국',
 '배터리',
 '업계',
 '무섭다',
 '약진',
 '고합',
 '역시',
 '삼성',
 '라든지',
 '업체',
 '종교',
 '배터리',
 '관련',
 '전기자동차',
 '배터리',
 '에너지',
 '저장장치',
 '배터리',
 '중국',
 '업체',
 '시장점유율',
 '확대',
 '상황',
 '문제',
 '배터리',
 '업체',
 '선반',
 '업체',
 '가어',
 '화학',
 '그다음',
 '참석',
 '고서',
 '이노베이션',
 '데이',
 '사가',
 '큰손',
 '요한',
 '변호사비용',
 '이세',
 '송정',
 '데이',
 '문제',
 '발단',
 '화학',
 '이노베이션',
 '이직',
 '화학',
 '친구',
 '영업비밀',
 '유출',
 '보고',
 '영업비밀',
 '침해',
 '미국',
 '국제무역',
 '위원회',
 '외부',
 '소송',
 '제기',
 '한국어',
 '지화학',
 '규명',
 '입장',
 '반하다',
 '이노베이션',
 '도그마',
 '소송',
 '특허',
 '침해',
 '이제우',
 '선반',
 '업체',
 '후반',
 '업체',
 '게정',
 '말초',
 '제소',
 '전이',
 '제이',
 '법적',
 '문제',
 '법정',
 '비용',
 '국

In [24]:
count = Counter(sample_clean.Contents_input[0])
words = dict(count.most_common())
words

{'사람': 57,
 '주식': 49,
 '생각': 39,
 '그렇다': 38,
 '좋다': 32,
 '투자': 29,
 '가지': 28,
 '많다': 24,
 '시작': 22,
 '공부': 21,
 '종목': 21,
 '이제': 21,
 '정말': 18,
 '도박': 18,
 '굉장하다': 17,
 '안되다': 16,
 '질투': 16,
 '경우': 16,
 '뭐': 16,
 '어렵다': 14,
 '그게': 14,
 '탐욕': 14,
 '시장': 13,
 '있다': 13,
 '잘못': 12,
 '투자자': 12,
 '뭔가': 11,
 '거기': 11,
 '정보': 11,
 '얘기': 11,
 '거지': 10,
 '친구': 10,
 '그것': 10,
 '진짜': 10,
 '좀더': 10,
 '거래': 10,
 '게이': 9,
 '프로': 9,
 '부분': 9,
 '대표': 9,
 '대해': 9,
 '보고': 9,
 '회사': 9,
 '그냥': 9,
 '좋아하다': 9,
 '나쁘다': 9,
 '아니다': 9,
 '주주': 9,
 '조급하다': 9,
 '같다': 9,
 '그다음': 8,
 '아예': 8,
 '마음': 8,
 '말씀': 8,
 '높다': 8,
 '방법': 8,
 '욕심': 8,
 '실수': 8,
 '쉽다': 8,
 '계속': 8,
 '상승': 7,
 '문제': 7,
 '그거': 7,
 '기본': 7,
 '가격': 7,
 '수익': 7,
 '가치투자': 7,
 '없다': 7,
 '결과': 6,
 '래서': 6,
 '이유': 6,
 '나름': 6,
 '애가': 6,
 '교육': 6,
 '실제': 6,
 '사고': 6,
 '자산': 6,
 '체크': 6,
 '테마주': 6,
 '반복': 6,
 '주가': 6,
 '저희': 6,
 '때문': 6,
 '게': 6,
 '심리': 6,
 '이렇다': 6,
 '이다': 5,
 '쉴드': 5,
 '아내': 5,
 '제안': 5,
 '투기': 5,
 '칠거지악': 5,
 '옛날': 5,
 '가깝다': 5,
 '아프다'

In [25]:
words_df = pd.DataFrame.from_dict([words]).T.reset_index()
words_df.columns = ['words', 'freq']

In [26]:
words_df

,words,freq
0,사람,57
1,주식,49
2,생각,39
3,그렇다,38
4,좋다,32
...,...,...
1764,운용,1
1765,해,1
1766,최준,1
1767,고맙다,1


In [64]:
words_df.to_csv('sample_wc.csv', header=True, index=False, encoding = 'utf-8-sig')

### 날짜별 버전

In [89]:
# lg chemical [날짜] before

# lgc_210908_bf = dataSet[dataSet['date_d'] == '2021-09-08']
# lgc_210908_bf

,st_n,st_cd,ch_nm,date,title,text,views,length,date_d,date_h,ud
0,LG화학,51910,삼프로TV_경제의신과함께,2021090800,[증시 염탐정] 약보합으로 출발한 시장.. 대형주의 방향은? f.이베스트투자증권 염...,자 그러면 오늘 또 증시 개장 상 1정리 해 주시아 염수 아니라 니모 지겠습니다 어...,100475,1813,2021-09-08,00,-1
1,LG화학,51910,삼프로TV_경제의신과함께,2021090800,"[뉴스3] ""美증시 비중축소""…잇단 비관론에 다우 270p↓",자 오늘 9월 8일 수요일 순서와먼저 뉴스 3부터 내 무게 좀 정리를해 드릴 텐데 ...,15483,1348,2021-09-08,00,-1
2,LG화학,51910,삼프로TV_경제의신과함께,2021090800,"[글로벌마켓 브리핑] 스가 총리 퇴진 발표 이후, 일본 증시 상승 랠리! f. 미래...",여러분 안녕하세요 광고 보고 왔습니다오늘 힘차게 한번 시작해 보도록하겠습니다 언제나...,7231,1711,2021-09-08,00,-1
3,LG화학,51910,삼프로TV_경제의신과함께,2021090800,[증시 셔터맨] 외인과 기관은 왜 삼성전자와 카카오뱅크를 매도할까? f. 한국투자증...,4 아 오늘한국투자증권 에서 오셨죠 우리박지영 차장을 모시겠습니다 어서오십시오 네 ...,21472,1511,2021-09-08,00,-1
4,LG화학,51910,삼프로TV_경제의신과함께,2021090800,[증시 셔터맨] 외인과 기관은 왜 삼성전자와 카카오뱅크를 매도할까? f. 한국투자증...,4 아 오늘한국투자증권 에서 오셨죠 우리박지영 차장을 모시겠습니다 어서오십시오 네 ...,21472,1511,2021-09-08,00,-1
5,LG화학,51910,삼프로TV_경제의신과함께,2021090800,"[글로벌마켓 브리핑] 스가 총리 퇴진 발표 이후, 일본 증시 상승 랠리! f. 미래...",여러분 안녕하세요 광고 보고 왔습니다오늘 힘차게 한번 시작해 보도록하겠습니다 언제나...,7231,1711,2021-09-08,00,-1
6,LG화학,51910,삼프로TV_경제의신과함께,2021090800,[증시 염탐정] 약보합으로 출발한 시장.. 대형주의 방향은? f.이베스트투자증권 염...,자 그러면 오늘 또 증시 개장 상 1정리 해 주시아 염수 아니라 니모 지겠습니다 어...,100475,1813,2021-09-08,00,-1
7,LG화학,51910,삼프로TV_경제의신과함께,2021090800,"[뉴스3] ""美증시 비중축소""…잇단 비관론에 다우 270p↓",자 오늘 9월 8일 수요일 순서와먼저 뉴스 3부터 내 무게 좀 정리를해 드릴 텐데 ...,15483,1348,2021-09-08,00,-1


In [141]:
## test

print(lgc_210908['Contents_input'][2][:20])

for i in range(len(lgc_210908['Contents_input'])):
    
    media = ['오늘', '년도', '이제', '요즘', '지금', '우리나라', '한번', '사실', '처럼', \
             '증시', '개장', '얘기', '이렇다', '참고', '없다', '때문', '아니다', \
             '안녕하십니까', '안녕하세요우', '안녕하다', '어서', '오십', '시오', '모습', '내용', '정도', \
             '그렇다', '어쨌든', '어떻다', '박지영', '미래에셋', '어머님', '음악']

    for word in lgc_210908['Contents_input'][i]:
#             if word in stopwords:
#                 lgc_210908['Contents_input'][i].remove(word)
        if word in media:
            lgc_210908['Contents_input'][i].remove(word)
                
print(lgc_210908['Contents_input'][2][:20])

['광고', '힘차다', '시작', '증권', '의장', '선임', '전이', '매년', '안녕하세요우', '난수', '간이', '럼또뭡', '사과', '감히', '방소', '점점', '덥다', '고합', '다정', '말로']
['광고', '힘차다', '시작', '증권', '의장', '선임', '전이', '매년', '난수', '간이', '럼또뭡', '사과', '감히', '방소', '점점', '덥다', '고합', '다정', '말로', '사과']


In [143]:
%%time

lgc_210908 = youtube_pre(lgc_210908_bf)

데이터 셋 구성 완료
okt 모듈 활성
태그 삭제 및 품사 태깅 완료
명사, 형용사 추출
불용어 사전 로딩..
불용어 처리 완료
단어 길이 1이하인 단어 제거
Wall time: 2min 31s


In [145]:
print(lgc_210908['Contents_input'][2][:20])

['광고', '힘차다', '시작', '미래에셋', '증권', '의장', '선임', '전이', '매년', '안녕하다', '안녕하세요우', '난수', '간이', '럼또뭡', '사과', '감히', '방소', '점점', '덥다', '고합']


In [20]:
lgc_210908.head()

NameError: name 'lgc_210908' is not defined

In [126]:
lgc_210908.Contents_input

0    [정리, 염수, 니모, 오십, 네마, 하든, 일로, 모습, 모집, 노래, 아이오와,...
1    [무게, 정리, 미국, 소식, 주너, 미국, 시장, 음소, 간밤, 진행, 좋다, 아...
2    [안녕하다, 광고, 힘차다, 시작, 처럼, 증권, 의장, 선임, 전이, 매년, 안녕...
3    [한국, 투자, 증권, 박지영, 차장, 시오, 안녕하십니까, 한국, 투자, 증권, ...
4    [한국, 투자, 증권, 박지영, 차장, 시오, 안녕하십니까, 한국, 투자, 증권, ...
5    [안녕하다, 광고, 힘차다, 시작, 처럼, 증권, 의장, 선임, 전이, 매년, 안녕...
6    [정리, 염수, 니모, 오십, 네마, 하든, 일로, 모습, 모집, 노래, 아이오와,...
7    [무게, 정리, 미국, 소식, 주너, 미국, 시장, 음소, 간밤, 진행, 좋다, 아...
Name: Contents_input, dtype: object

In [127]:
lgc_210908.Contents_input[0]

['정리',
 '염수',
 '니모',
 '오십',
 '네마',
 '하든',
 '일로',
 '모습',
 '모집',
 '노래',
 '아이오와',
 '따갑다',
 '달라',
 '위험부담',
 '예맥',
 '강하다',
 '요청',
 '김택진',
 '진영',
 '머신',
 '매트',
 '개장',
 '상황',
 '은행',
 '관심',
 '좋다',
 '금융',
 '재정',
 '옥애널',
 '리스트',
 '은행',
 '업종',
 '간격',
 '보고',
 '간단',
 '월동',
 '외국인',
 '매도',
 '계속',
 '코스피',
 '은행',
 '수급',
 '무료',
 '좋다',
 '목요일',
 '그다음',
 '긴축',
 '속도',
 '경기',
 '확장',
 '북면',
 '연장',
 '가능성',
 '다시',
 '경기',
 '금리',
 '베팅',
 '실적',
 '상황',
 '서주',
 '당대',
 '금도',
 '가능성',
 '투자',
 '수요',
 '증가',
 '가능성',
 '고합',
 '다그',
 '래서',
 '변경',
 '은행',
 '전체',
 '순위',
 '좋다',
 '작년',
 '대비',
 '가내',
 '순위',
 '증가',
 '예상',
 '그다음',
 '올해',
 '예상',
 '배당',
 '수익률',
 '투자',
 '행주',
 '연말',
 '은행',
 '대해',
 '관심',
 '좀뭐',
 '리스크',
 '만화',
 '접근',
 '요좀',
 '해주시',
 '좋다',
 '그다음',
 '증권',
 '재운',
 '이분',
 '굉장하다',
 '피트',
 '코인',
 '보고서',
 '비트코인',
 '가기',
 '급락',
 '최근',
 '설번',
 '거기',
 '법정',
 '놈',
 '개널',
 '비트코인',
 '법정',
 '통화',
 '채택',
 '예고',
 '살바도르',
 '달러',
 '버팔로',
 '서치',
 '인정',
 '시작',
 '제품',
 '구매',
 '세금',
 '납부',
 '비트코인',
 '자국',
 '국민',
 '혼란',
 '위해',
 '비트코인',
 '달러',


In [128]:
lgc_210908_list = []

for i in lgc_210908.Contents_input:
    for j in i:
        lgc_210908_list.append(j)
    
lgc_210908_list

['정리',
 '염수',
 '니모',
 '오십',
 '네마',
 '하든',
 '일로',
 '모습',
 '모집',
 '노래',
 '아이오와',
 '따갑다',
 '달라',
 '위험부담',
 '예맥',
 '강하다',
 '요청',
 '김택진',
 '진영',
 '머신',
 '매트',
 '개장',
 '상황',
 '은행',
 '관심',
 '좋다',
 '금융',
 '재정',
 '옥애널',
 '리스트',
 '은행',
 '업종',
 '간격',
 '보고',
 '간단',
 '월동',
 '외국인',
 '매도',
 '계속',
 '코스피',
 '은행',
 '수급',
 '무료',
 '좋다',
 '목요일',
 '그다음',
 '긴축',
 '속도',
 '경기',
 '확장',
 '북면',
 '연장',
 '가능성',
 '다시',
 '경기',
 '금리',
 '베팅',
 '실적',
 '상황',
 '서주',
 '당대',
 '금도',
 '가능성',
 '투자',
 '수요',
 '증가',
 '가능성',
 '고합',
 '다그',
 '래서',
 '변경',
 '은행',
 '전체',
 '순위',
 '좋다',
 '작년',
 '대비',
 '가내',
 '순위',
 '증가',
 '예상',
 '그다음',
 '올해',
 '예상',
 '배당',
 '수익률',
 '투자',
 '행주',
 '연말',
 '은행',
 '대해',
 '관심',
 '좀뭐',
 '리스크',
 '만화',
 '접근',
 '요좀',
 '해주시',
 '좋다',
 '그다음',
 '증권',
 '재운',
 '이분',
 '굉장하다',
 '피트',
 '코인',
 '보고서',
 '비트코인',
 '가기',
 '급락',
 '최근',
 '설번',
 '거기',
 '법정',
 '놈',
 '개널',
 '비트코인',
 '법정',
 '통화',
 '채택',
 '예고',
 '살바도르',
 '달러',
 '버팔로',
 '서치',
 '인정',
 '시작',
 '제품',
 '구매',
 '세금',
 '납부',
 '비트코인',
 '자국',
 '국민',
 '혼란',
 '위해',
 '비트코인',
 '달러',


In [129]:
len(lgc_210908_list)

9458

In [130]:
count = Counter(lgc_210908_list)
words = dict(count.most_common())

words_df = pd.DataFrame.from_dict([words]).T.reset_index()
words_df.columns = ['words', 'freq']

In [131]:
words_df

,words,freq
0,좋다,124
1,중국,102
2,이제,78
3,기업,56
4,굉장하다,54
...,...,...
2246,요,2
2247,키스,2
2248,깔끔하다,2
2249,반값,2


In [123]:
words_df.to_csv('lgc_210908_wc.csv', header=True, index=False, encoding = 'utf-8-sig')